<a href="https://colab.research.google.com/github/katyk20/DistanceLearning_Capstone/blob/main/Code%20Notebooks/Model_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np


In [3]:
# ! git clone https://github.com/katyk20/DistanceLearning_Capstone

In [4]:
#read in file that crosswalks IPEDS CIP codes with BLS SOC codes
#Extract code columns using sheet SOC-CIP
crosswalk=pd.read_excel("/content/drive/MyDrive/Capstone_Winter2024/Data Sources/Bureau of Labor Statistics/CIP2020_SOC2018_Crosswalk.xlsx", sheet_name="SOC-CIP")


In [5]:
# # crosswalk.head()
cols_keep =['SOC2018Code','CIP2020Code']
codes = crosswalk[cols_keep]
codes= codes.rename(columns ={'CIP2020Code':'CIPCODE'})


In [6]:
codes.head()
# # len(codes)

,SOC2018Code,CIPCODE
0,11-1011,44.0401
1,11-1011,52.0101
2,11-1011,52.0201
3,11-1011,52.0206
4,11-1011,52.0701


In [7]:
#read in c2022dep_DATA degrees conferred/completions
completes22 = pd.read_csv('/content/drive/MyDrive/Capstone_Winter2024/Data Sources/NCES IPEDS 2019-2022 Data/c2022dep_DATA.csv')

In [8]:
# completes22.head(20)

In [9]:

# Convert CIP float values to strings and remove institutions by summary CIPS ending with  ".0"
mask = completes22['CIPCODE'].astype(str).str.endswith('.0')
summarycips = completes22[mask]
completes22=completes22[~mask]


In [10]:
# summarycips.head()
completes22.head()

,UNITID,CIPCODE,PTOTAL,PTOTALDE,PTOTALDES,PASSOC,PASSOCDE,PASSOCDES,PBACHL,PBACHLDE,...,PCERT2DES,PCERT4,PCERT4DE,PCERT4DES,PPBACC,PPBACCDE,PPBACCDES,PPMAST,PPMASTDE,PPMASTDES
1,100654,1.0999,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,100654,1.1001,3,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,100654,1.9999,3,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,100654,3.0599,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,100654,4.0301,2,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# unique_cips =sorted(completes22['CIPCODE'].unique())
# unique_cips

In [12]:
# #read in BLS 2022 Emp major occupation file Table 1.1
bls22= pd.read_excel('/content/drive/MyDrive/Capstone_Winter2024/Data Sources/Bureau of Labor Statistics/1.7EmpDetailedOccWorkerChar2022.xlsx', header = 1)


In [13]:
#columns to keep
cols_keep = (['2022 National Employment Matrix title',
       '2022 National Employment Matrix code',
       'Employment, 2022', 'Employment, 2032',
       'Employment change, numeric, 2022-32',
       'Employment change, percent, 2022-32', 'Percent self employed, 2022',
       'Occupational openings, 2022-32 annual average',
       'Median annual wage, 2022(1)'])
bls22 = bls22[cols_keep]
# bls22.tail()

In [14]:
bls22 = bls22.dropna().drop([0])

In [15]:
#occ22.columns
#rename columns
bls22 = bls22.rename(columns ={'2022 National Employment Matrix title': 'OCC TITLE 22',
                               '2022 National Employment Matrix code': 'SOC2018Code',
                          'Employment, 2022': 'EMP 2022',
                          'Employment, 2032': 'EMP 2032',
                          'Employment change, numeric, 2022-32':'EMP CHANGE NUM',
                          'Employment change, percent, 2022-32': 'EMP CHANGE 22-32 %',
                          'Percent self employed, 2022' : 'SELF EMP %',
                          'Occupational openings, 2022-32 annual average': 'OCC OPEN ANN AVG 22-32',
                          'Median annual wage, 2022(1)': 'MED ANN WAGE 2022'})

In [16]:
bls22.tail()

,OCC TITLE 22,SOC2018Code,EMP 2022,EMP 2032,EMP CHANGE NUM,EMP CHANGE 22-32 %,SELF EMP %,OCC OPEN ANN AVG 22-32,MED ANN WAGE 2022
1108,"Pump operators, except wellhead pumpers",53-7072,11.2,11.6,0.4,3.6,—,1.2,55860
1109,Wellhead pumpers,53-7073,16.2,15.9,-0.4,-2.2,0.4,1.6,63280
1110,Refuse and recyclable material collectors,53-7081,146.4,151.1,4.7,3.2,7.6,18.6,43540
1111,"Tank car, truck, and ship loaders",53-7121,13.3,13.5,0.2,1.6,3.3,1.6,53930
1112,"Material moving workers, all other",53-7199,26.1,27.0,0.9,3.5,3.1,3.2,38800


In [17]:
#merge bls labor data with codes crosswalk data on common SOC2018Code
merged_bls22= pd.merge(bls22, codes, on ='SOC2018Code', how = 'inner')

In [18]:
# merged_bls22.head()

In [19]:
#merge merged_bls22 with completes22 dataset based on common CIPCODE
bls_completes22=pd.merge(completes22, merged_bls22, on='CIPCODE', how ='inner')

In [20]:
# bls_completes22.columns
"""['UNITID', 'CIPCODE', 'PTOTAL', 'PTOTALDE', 'PTOTALDES', 'PASSOC',
       'PASSOCDE', 'PASSOCDES', 'PBACHL', 'PBACHLDE', 'PBACHLDES', 'PMASTR',
       'PMASTRDE', 'PMASTRDES', 'PDOCRS', 'PDOCRSDE', 'PDOCRSDES', 'PDOCPP',
       'PDOCPPDE', 'PDOCPPDES', 'PDOCOT', 'PDOCOTDE', 'PDOCOTDES', 'PCERT1A',
       'PCERT1ADE', 'PCERT1ADES', 'PCERT1B', 'PCERT1BDE', 'PCERT1BDES',
       'PCERT2', 'PCERT2DE', 'PCERT2DES', 'PCERT4', 'PCERT4DE', 'PCERT4DES',
       'PPBACC', 'PPBACCDE', 'PPBACCDES', 'PPMAST', 'PPMASTDE', 'PPMASTDES  ',
       'OCC TITLE 22', 'SOC2018Code', 'EMP 2022', 'EMP 2032', 'EMP CHANGE NUM',
       'EMP CHANGE 22-32 %', 'SELF EMP %', 'OCC OPEN ANN AVG 22-32',
       'MED ANN WAGE 2022']"""

"['UNITID', 'CIPCODE', 'PTOTAL', 'PTOTALDE', 'PTOTALDES', 'PASSOC',\n       'PASSOCDE', 'PASSOCDES', 'PBACHL', 'PBACHLDE', 'PBACHLDES', 'PMASTR',\n       'PMASTRDE', 'PMASTRDES', 'PDOCRS', 'PDOCRSDE', 'PDOCRSDES', 'PDOCPP',\n       'PDOCPPDE', 'PDOCPPDES', 'PDOCOT', 'PDOCOTDE', 'PDOCOTDES', 'PCERT1A',\n       'PCERT1ADE', 'PCERT1ADES', 'PCERT1B', 'PCERT1BDE', 'PCERT1BDES',\n       'PCERT2', 'PCERT2DE', 'PCERT2DES', 'PCERT4', 'PCERT4DE', 'PCERT4DES',\n       'PPBACC', 'PPBACCDE', 'PPBACCDES', 'PPMAST', 'PPMASTDE', 'PPMASTDES  ',\n       'OCC TITLE 22', 'SOC2018Code', 'EMP 2022', 'EMP 2032', 'EMP CHANGE NUM',\n       'EMP CHANGE 22-32 %', 'SELF EMP %', 'OCC OPEN ANN AVG 22-32',\n       'MED ANN WAGE 2022']"

In [21]:
# bls_completes22.tail(50)

In [22]:
#read in distance enrollment dataset = depenrollment18_22.csv
enrolled = pd.read_csv('/content/drive/MyDrive/Capstone_Winter2024/Final Datasets_Katy/dep_enrollment18_22.csv')

In [23]:
#filter for Year =2022 and EFDELEV = 1 (Summary line for institution)
# year = ['2022']
enrolled_22 = enrolled[(enrolled['Year']==2022) & (enrolled['EFDELEV'] == 1)]
enrolled_22.head()

,UNITID,INSTNM,Year,IALIAS,STABBR,OBEREG,ICLEVEL,CONTROL,HLOFFER,DEGGRANT,...,EFDELEV,EFDETOT,EFDEEXC,EFDESOM,EFDENON,EFDEEX1,EFDEEX2,EFDEEX3,EFDEEX4,EFDEEX5
89561,100654,Alabama A & M University,2022,AAMU,AL,5,1,1,9,1,...,1,6007.0,429.0,3269.0,2309.0,284.0,57.0,37.0,0.0,51.0
89566,100663,University of Alabama at Birmingham,2022,UAB,AL,5,1,1,9,1,...,1,21639.0,5984.0,8687.0,6968.0,4199.0,1762.0,0.0,23.0,0.0
89571,100690,Amridge University,2022,Southern Christian University Regions University,AL,5,1,2,9,1,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89575,100706,University of Alabama in Huntsville,2022,UAH University of Alabama Huntsville,AL,5,1,1,9,1,...,1,9237.0,1177.0,4507.0,3553.0,953.0,210.0,11.0,3.0,0.0
89580,100724,Alabama State University,2022,,AL,5,1,1,9,1,...,1,3828.0,420.0,2413.0,995.0,325.0,86.0,7.0,0.0,2.0


In [24]:
#if EFDETOT is NAN (Not applicable)- DROP those rows
enrolled_22 = enrolled_22.dropna()
len(enrolled_22)

3071

In [25]:
enrolled_22.columns
'''['UNITID', 'INSTNM', 'Year', 'IALIAS', 'STABBR', 'OBEREG', 'ICLEVEL',
       'CONTROL', 'HLOFFER', 'DEGGRANT', 'PSEFLAG', 'LONGITUD', 'LATITUDE',
       'EFDELEV', 'EFDETOT', 'EFDEEXC', 'EFDESOM', 'EFDENON', 'EFDEEX1',
       'EFDEEX2', 'EFDEEX3', 'EFDEEX4', 'EFDEEX5 '],'''

"['UNITID', 'INSTNM', 'Year', 'IALIAS', 'STABBR', 'OBEREG', 'ICLEVEL',\n       'CONTROL', 'HLOFFER', 'DEGGRANT', 'PSEFLAG', 'LONGITUD', 'LATITUDE',\n       'EFDELEV', 'EFDETOT', 'EFDEEXC', 'EFDESOM', 'EFDENON', 'EFDEEX1',\n       'EFDEEX2', 'EFDEEX3', 'EFDEEX4', 'EFDEEX5 '],"

In [31]:
#columns to merge into bls_completes22
#variables of interest to start are distance education exclusive, some DE, and no DE enrollments
#EXE1=exclusively in de courses/same state
#EXE2=exclusively in de course/US but not same state
#EXE4= exclusively in de course/outside US
cols_to_merge=['UNITID','STABBR','OBEREG', 'EFDEEXC', 'EFDESOM', 'EFDENON',
              'EFDEEX1', 'EFDEEX2', 'EFDEEX4']

merged_df = pd.merge(bls_completes22, enrolled_22[cols_to_merge], on='UNITID', how='inner')
#add year subscript
year = '22'
cols_to_merge2=['PTOTAL', 'PTOTALDE', 'PTOTALDES', 'PASSOC',
       'PASSOCDE', 'PASSOCDES', 'PBACHL', 'PBACHLDE', 'PBACHLDES', 'PMASTR',
       'PMASTRDE', 'PMASTRDES', 'PDOCRS', 'PDOCRSDE', 'PDOCRSDES', 'PDOCPP',
       'PDOCPPDE', 'PDOCPPDES', 'PDOCOT', 'PDOCOTDE', 'PDOCOTDES', 'PCERT1A',
       'PCERT1ADE', 'PCERT1ADES', 'PCERT1B', 'PCERT1BDE', 'PCERT1BDES',
       'PCERT2', 'PCERT2DE', 'PCERT2DES', 'PCERT4', 'PCERT4DE', 'PCERT4DES',
       'PPBACC', 'PPBACCDE', 'PPBACCDES', 'PPMAST', 'PPMASTDE', 'PPMASTDES',
       'EFDEEXC', 'EFDESOM', 'EFDENON', 'EFDEEX1', 'EFDEEX2', 'EFDEEX4']

renamed_columns = {col: f"{col}_{year}" for col in cols_to_merge2}
merged_df = merged_df.rename(columns=renamed_columns)

merged_df.columns

Index(['UNITID', 'CIPCODE', 'PTOTAL_22', 'PTOTALDE_22', 'PTOTALDES_22',
       'PASSOC_22', 'PASSOCDE_22', 'PASSOCDES_22', 'PBACHL_22', 'PBACHLDE_22',
       'PBACHLDES_22', 'PMASTR_22', 'PMASTRDE_22', 'PMASTRDES_22', 'PDOCRS_22',
       'PDOCRSDE_22', 'PDOCRSDES_22', 'PDOCPP_22', 'PDOCPPDE_22',
       'PDOCPPDES_22', 'PDOCOT_22', 'PDOCOTDE_22', 'PDOCOTDES_22',
       'PCERT1A_22', 'PCERT1ADE_22', 'PCERT1ADES_22', 'PCERT1B_22',
       'PCERT1BDE_22', 'PCERT1BDES_22', 'PCERT2_22', 'PCERT2DE_22',
       'PCERT2DES_22', 'PCERT4_22', 'PCERT4DE_22', 'PCERT4DES_22', 'PPBACC_22',
       'PPBACCDE_22', 'PPBACCDES_22', 'PPMAST_22', 'PPMASTDE_22',
       'PPMASTDES  ', 'OCC TITLE 22', 'SOC2018Code', 'EMP 2022', 'EMP 2032',
       'EMP CHANGE NUM', 'EMP CHANGE 22-32 %', 'SELF EMP %',
       'OCC OPEN ANN AVG 22-32', 'MED ANN WAGE 2022', 'STABBR', 'OBEREG',
       'EFDEEXC_22', 'EFDESOM_22', 'EFDENON_22', 'EFDEEX1_22', 'EFDEEX2_22',
       'EFDEEX4_22'],
      dtype='object')

In [32]:
#columns to keep
#degrees total and those by DE excusive, DE some, employment data, and year enrollment data

cols_keep=['UNITID','STABBR', 'OBEREG', 'PTOTAL_22', 'PTOTALDE_22', 'PTOTALDES_22',
       'PASSOC_22', 'PASSOCDE_22', 'PASSOCDES_22', 'PBACHL_22', 'PBACHLDE_22',
       'PBACHLDES_22', 'PMASTR_22', 'PMASTRDE_22', 'PMASTRDES_22', 'PDOCRS_22',
       'PDOCRSDE_22', 'PDOCRSDES_22', 'PDOCPP_22', 'PDOCPPDE_22',
       'PDOCPPDES_22', 'PDOCOT_22', 'PDOCOTDE_22', 'PDOCOTDES_22',
       'PCERT1A_22', 'PCERT1ADE_22', 'PCERT1ADES_22', 'PCERT1B_22',
       'PCERT1BDE_22', 'PCERT1BDES_22', 'PCERT2_22', 'PCERT2DE_22',
       'PCERT2DES_22', 'PCERT4_22', 'PCERT4DE_22', 'PCERT4DES_22', 'PPBACC_22',
       'PPBACCDE_22', 'PPBACCDES_22', 'PPMAST_22', 'PPMASTDE_22',
       'PPMASTDES  ',  'EMP 2022', 'EMP 2032',
       'EMP CHANGE NUM', 'EMP CHANGE 22-32 %', 'SELF EMP %',
       'OCC OPEN ANN AVG 22-32', 'MED ANN WAGE 2022',
       'EFDEEXC_22', 'EFDESOM_22', 'EFDENON_22', 'EFDEEX1_22', 'EFDEEX2_22',
       'EFDEEX4_22']

merged_df= merged_df[cols_keep]
merged_df.tail()
# len(merged_df)#640594
# merged_df.isnull().sum()

,UNITID,STABBR,OBEREG,PTOTAL_22,PTOTALDE_22,PTOTALDES_22,PASSOC_22,PASSOCDE_22,PASSOCDES_22,PBACHL_22,...,EMP CHANGE 22-32 %,SELF EMP %,OCC OPEN ANN AVG 22-32,MED ANN WAGE 2022,EFDEEXC_22,EFDESOM_22,EFDENON_22,EFDEEX1_22,EFDEEX2_22,EFDEEX4_22
640589,488907,NV,8,1,0,0,0,0,0,0,...,2.0,8.4,2.0,106230,29.0,1.0,0.0,29.0,0.0,0.0
640590,490106,VA,5,4,0,0,0,0,0,0,...,5.3,69.4,1.8,72220,165.0,191.0,37.0,62.0,97.0,2.0
640591,490106,VA,5,4,0,0,0,0,0,0,...,2.0,8.4,2.0,106230,165.0,191.0,37.0,62.0,97.0,2.0
640592,117520,CA,8,1,0,0,0,0,0,0,...,19.1,0.2,27.4,100300,14.0,500.0,73.0,7.0,7.0,0.0
640593,117520,CA,8,1,0,0,0,0,0,0,...,8.8,31.1,2.6,75380,14.0,500.0,73.0,7.0,7.0,0.0


In [41]:
#filter enrollment data 2018-2022

def filter_enrolled(enrolled_df, year, efdelev):
    filtered_df = enrolled_df[(enrolled_df['Year'] == year) & (enrolled_df['EFDELEV'] == efdelev)]
    filtered_df = filtered_df.dropna()
    cols_to_keep=['UNITID', 'EFDEEXC', 'EFDESOM', 'EFDENON',
              'EFDEEX1', 'EFDEEX2', 'EFDEEX4']
    return filtered_df[cols_to_keep]


enrolled_18 = filter_enrolled(enrolled, 2018, 1)
enrolled_19 = filter_enrolled(enrolled, 2019, 1)
enrolled_20 = filter_enrolled(enrolled, 2020, 1)
enrolled_21 = filter_enrolled(enrolled, 2021, 1)

enrolled_21

,UNITID,EFDEEXC,EFDESOM,EFDENON,EFDEEX1,EFDEEX2,EFDEEX4
67092,100654,426.0,2513.0,3030.0,276.0,125.0,0.0
67097,100663,6460.0,8202.0,7627.0,4315.0,2113.0,4.0
67106,100706,1324.0,5117.0,3195.0,1074.0,237.0,3.0
67111,100724,644.0,2783.0,537.0,484.0,156.0,0.0
67116,100751,6086.0,9339.0,22891.0,3291.0,2644.0,88.0
...,...,...,...,...,...,...,...
89405,496186,121.0,0.0,0.0,117.0,4.0,0.0
89427,496469,142.0,0.0,0.0,10.0,28.0,104.0
89502,496973,23.0,6.0,39.0,22.0,1.0,0.0
89508,497000,192.0,47.0,13.0,14.0,45.0,0.0


In [42]:
#merge 2018-2021 enrollment data into merged_df on UNITID

year_mapping = {
     'enrolled_18': 18,
     'enrolled_19': 19,
     'enrolled_20': 20,
     'enrolled_21': 21,
}

datasets_dict = {
    'enrolled_18': enrolled_18,
    'enrolled_19': enrolled_19,
    'enrolled_20': enrolled_20,
    'enrolled_21': enrolled_21,
}
column_name = 'UNITID'

def enroll_merge(merged_df,datasets, year_mapping, column_name):

  # Merge each dataset one by one, adding subscript to column names indicating the year of the dataset
  for dataset_name, df in datasets.items():
    year = year_mapping.get(dataset_name)
    if year is None:
      raise ValueError(f"Year information not found for dataset '{dataset_name}'.")
    # Add subscript to column names indicating the year of the dataset
    df.columns = [f"{col}_{year}" if col != column_name else col for col in df.columns]

  #   # Merge the dataset with the existing merged DataFrame
  # merged_df = pd.merge(merged_df, df, on=column_name, how='inner')

  # Merge the dataset with the merged DataFrame
    if merged_df is None:
       merged_df = df.copy()
    else:
      merged_df = pd.merge(merged_df, df, on=column_name, how='outer')

  return merged_df

model_data= enroll_merge(merged_df,datasets_dict, year_mapping,column_name)


In [43]:
# #rearrange columns
cols= ['EFDEEXC_22', 'EFDESOM_22', 'EFDENON_22',
       'EFDEEX1_22', 'EFDEEX2_22', 'EFDEEX4_22']

def move_cols_to_end(df, cols):
    return df[[x for x in df.columns if not x in cols] + cols]

model_data = move_cols_to_end(model_data, cols)



In [44]:
model_data.columns

Index(['UNITID', 'STABBR', 'OBEREG', 'PTOTAL_22', 'PTOTALDE_22',
       'PTOTALDES_22', 'PASSOC_22', 'PASSOCDE_22', 'PASSOCDES_22', 'PBACHL_22',
       'PBACHLDE_22', 'PBACHLDES_22', 'PMASTR_22', 'PMASTRDE_22',
       'PMASTRDES_22', 'PDOCRS_22', 'PDOCRSDE_22', 'PDOCRSDES_22', 'PDOCPP_22',
       'PDOCPPDE_22', 'PDOCPPDES_22', 'PDOCOT_22', 'PDOCOTDE_22',
       'PDOCOTDES_22', 'PCERT1A_22', 'PCERT1ADE_22', 'PCERT1ADES_22',
       'PCERT1B_22', 'PCERT1BDE_22', 'PCERT1BDES_22', 'PCERT2_22',
       'PCERT2DE_22', 'PCERT2DES_22', 'PCERT4_22', 'PCERT4DE_22',
       'PCERT4DES_22', 'PPBACC_22', 'PPBACCDE_22', 'PPBACCDES_22', 'PPMAST_22',
       'PPMASTDE_22', 'PPMASTDES  ', 'EMP 2022', 'EMP 2032', 'EMP CHANGE NUM',
       'EMP CHANGE 22-32 %', 'SELF EMP %', 'OCC OPEN ANN AVG 22-32',
       'MED ANN WAGE 2022', 'EFDEEXC_18', 'EFDESOM_18', 'EFDENON_18',
       'EFDEEX1_18', 'EFDEEX2_18', 'EFDEEX4_18', 'EFDEEXC_19', 'EFDESOM_19',
       'EFDENON_19', 'EFDEEX1_19', 'EFDEEX2_19', 'EFDEEX4_19', 'E

In [27]:
#merge in unemployment rate on STATE ABBR
ur=pd.read_excel('/content/drive/MyDrive/Capstone_Winter2024/Data Sources/Bureau of Labor Statistics/StateUR18_22.xlsx')

In [45]:
col_change={'State':'STABBR',
              '2018 rate': 'ST_UR18',
             '2019 rate': 'ST_UR19',
             '2020 rate': 'ST_UR20',
             '2021 rate': 'ST_UR21',
             '2022 rate': 'ST_UR22'}
ur = ur.rename(columns=col_change)
ur.head()


,STABBR,ST_UR18,ST_UR19,ST_UR20,ST_UR21,ST_UR22
0,AL,3.9,3.2,6.4,3.4,2.5
1,AK,6.0,5.6,8.3,6.4,4.2
2,AZ,4.8,4.8,7.8,5.1,3.8
3,AR,3.7,3.5,6.2,4.0,3.2
4,CA,4.2,4.1,10.1,7.3,4.3


In [47]:
def ur_merge(df1, df2, column_name):
  merged_df =pd.merge(df1, df2, on = column_name, how= "inner" )
  return merged_df

model_data2=ur_merge(model_data, ur, "STABBR")

In [48]:
model_data2.head()

,UNITID,STABBR,OBEREG,PTOTAL_22,PTOTALDE_22,PTOTALDES_22,PASSOC_22,PASSOCDE_22,PASSOCDES_22,PBACHL_22,...,EFDESOM_22,EFDENON_22,EFDEEX1_22,EFDEEX2_22,EFDEEX4_22,ST_UR18,ST_UR19,ST_UR20,ST_UR21,ST_UR22
0,100654,AL,5.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,3269.0,2309.0,284.0,57.0,0.0,3.9,3.2,6.4,3.4,2.5
1,100654,AL,5.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,3269.0,2309.0,284.0,57.0,0.0,3.9,3.2,6.4,3.4,2.5
2,100654,AL,5.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,3269.0,2309.0,284.0,57.0,0.0,3.9,3.2,6.4,3.4,2.5
3,100654,AL,5.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,3269.0,2309.0,284.0,57.0,0.0,3.9,3.2,6.4,3.4,2.5
4,100654,AL,5.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,3269.0,2309.0,284.0,57.0,0.0,3.9,3.2,6.4,3.4,2.5


In [ ]:
# model_data.to_csv('model_data1.csv', index=False)